In [11]:
import os
import cv2 
import sys
import numpy as np

In [12]:
class HTMLComponent:
    def  __init__(self,img,x,y,h,w):
        self.img=img
        self.x=x
        self.y=y
        self.h=h
        self.w=w
        self.sub=[]#sub elements
    def setImage(self,img):
        self.img=img
    def setCoordinates(self,x,y):
        self.x=x
        self.y=y
    def AddSubElement(self,e):
        self.sub.append(e);
    def getSubElements(self):
        return self.sub;
    def getImage(self):
        return self.img
    def getCorrdinates(self):
        return self.x,self.y
    def getAttributes(self):
        return self.x,self.y,self.w,self.h
    
        
    
    

In [13]:
class Webpage:
    def __init__(self):
        self.elements=[]
    def addElement(self,e):
        self.elements.append(e);
    def getelements(self):
        return self.elements
    def setElements(e):
        self.elements=e

In [14]:
class HtmlMapper:
    def ImgToWebpage(self,image):
        webpage=Webpage()
        #img=image.copy()
        img=self.getBoundariesEnchanced(image.copy())
        
        img=self.EnhanceInnerSurface(img.copy())
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)  
        ret,img = cv2.threshold(gray, 0, 255,cv2.THRESH_OTSU|cv2.THRESH_BINARY_INV)
        height, width = img.shape[:2]
        cv2.imwrite("contour_enchanced.png",img)
        #gray=cv2.cvtColor(image.copy(),cv2.COLOR_BGR2GRAY)
        edged = cv2.Canny(img, 10, 20)
        #gray = cv2.cvtColor(edged, cv2.COLOR_BGR2GRAY)  
        ret,edged = cv2.threshold(gray, 0, 255,cv2.THRESH_OTSU|cv2.THRESH_BINARY_INV)
        cv2.imshow("canny2nd",edged)
        cv2.waitKey();
        #edged=img
        #edged=cv2.cvtColor(edged,cv2.COLOR_BGR2GRAY)
        (cnts, h) = cv2.findContours(edged.copy(), cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
        idx = 0
        
        parentElement=HTMLComponent(-1,-1,-1,-1,-1)
        for c,h1 in zip(cnts,h[0]):
            x,y,w,h = cv2.boundingRect(c)
            if w>15 and h>15:# and x+w<700 and y+h<700:#15
                idx+=1
                new_img=image[y:y+h,x:x+w]
                element=HTMLComponent(new_img,x,y,h,w)
                if(h1[3]!=-1):
                    #if new image is less than 80% of parent image
                    pshape=parentElement.getImage().shape
                    newshape=new_img.shape
                    if(0.4*pshape[0]*pshape[1]>newshape[0]*newshape[1]):
                        parentElement.AddSubElement(element)
                else:
                    parentElement=element;
                    webpage.addElement(element)
                    
        #cv2.imshow('final', minus_img)
        #cv2.waitKey()
        return webpage  
    def MapHtml(self,webpage,path):
        code=""
        index=0
        for e in webpage.getelements():
            path = "../images/"
            ipath=path+str(index)+".png"
            cv2.imwrite(path+str(index)+".png",e.getImage())
            x,y,w,h=e.getAttributes()
            for (i,e1) in enumerate(e.getSubElements()):
                path1=path+str(index)+"-"+str(i)+".png"
                cv2.imwrite(path1,e1.getImage())
                x1,y1,w1,h1=e1.getAttributes()
                code+="<IMG STYLE=\"position:absolute; TOP:"+str(y1)+"px;LEFT:"+str(x1)+"px; WIDTH:"+str(w1)+"px; HEIGHT:"+str(h1)+"px\" SRC=\""+path1+"\">"
            #code+="<IMG STYLE=\"position:absolute; TOP:"+str(y)+"px;LEFT:"+str(x)+"px; WIDTH:"+str(w)+"px; HEIGHT:"+str(h)+"px\" SRC=\""+ipath+"\">"
            index+=1

        return code
    def getBoundariesEnchanced(self,img):
        
        #img = cv2.imread('stack.png')
        image = cv2.cvtColor(img.copy(),cv2.COLOR_BGR2GRAY)
        edges = cv2.Canny(img.copy(),10,20)
        cv2.imshow("canny",edges);
        cv2.waitKey();
        #ret,edges = cv2.threshold(image.copy(), 0, 255,cv2.THRESH_OTSU|cv2.THRESH_BINARY_INV)
        cv2.imwrite("canny.png",edges)#output file
        #ab = cv2.imread('canny.png')
        #ab = cv2.cvtColor(ab,cv2.COLOR_BGR2GRAY)

        contours, hierarchy = cv2.findContours(edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        #print ('no of shapes {0}'.format(len(contours)))
        # for cnt in contours:
        #     rect = cv2.minAreaRect(cnt)
        #     box = cv2.boxPoints(rect)
        #     box = np.int0(box)
        #     img = cv2.drawContours(img, [box], 0, (0,255,0), 3)
        cntlist=[]
        for cnt in contours:
            #epsilon = 0.01*cv2.arcLength(cnt, True)
            #approx = cv2.approxPolyDP(cnt, epsilon, True)
            x,y,w,h = cv2.boundingRect(cnt)
            if w>15 and h>15:# and w<width and h<height:
                cv2.rectangle(img, (x, y), (x + w, y + h), (0, 0, 0), 2)
            #img = cv2.drawContours(img, [approx], 0, (0,255,0),1)
        cv2.imwrite("canny.png",img)
        return img

    def ImgToHtml(self,image,path):
        
        w=self.ImgToWebpage(image)
        s=self.MapHtml(w,path)
        return s;
    def EnhanceInnerSurface(self,img):
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)  
        ret,thresh1 = cv2.threshold(gray, 0, 255,cv2.THRESH_OTSU|cv2.THRESH_BINARY_INV)
       
        rect_kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (15, 8))
        dilation = cv2.dilate(thresh1, rect_kernel, iterations = 1)
        cv2.imshow("otsu",thresh1);
        cv2.imshow("dilation",dilation);
        cv2.waitKey();
        contours, hierarchy = cv2.findContours(dilation, cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)
        im2 = img.copy()
        for cnt in contours:
            x, y, w, h = cv2.boundingRect(cnt)
            if(h>15 and w>15):
                cv2.rectangle(im2, (x, y), (x + w, y + h), (0, 255, 0), 2)
        #return thresh1
        cv2.imwrite("otso.png",im2)
        return im2

In [15]:
def main(arg):
    #p=arg[0]
    #imgname=arg[1]
    p="../generated_resources/webpages/"
    imgname="../resources/stack.png"#change this with the path of image
    i = cv2.imread(imgname)
    h=HtmlMapper()
    code=h.ImgToHtml(i,p)
    file=open(p+"webpage.html","w+")
    file.write(code)
    file.close()
    os.system("start \"\" "+p+"webpage.html\"")
if __name__ == "__main__":
    main(sys.argv[1:])  
    
        